In [1]:
import pandas as pd
from rdkit.Chem.PandasTools import LoadSDF
import pubchempy as pcp
from tqdm import tqdm
from chembl_webresource_client.new_client import new_client
import numpy as np

In [2]:
data = pd.read_csv('chembl.csv', sep=';', error_bad_lines=True)
withdrawn = LoadSDF('withdrawn_withdrawn.sdf')

* Download Pubchem for Chembl

* Process chembl:
    1. Drop polymers
    2. Drop compounds without smiles

In [3]:
data = data.loc[data['Drug Type'] != "10:Polymer"]
data = data.loc[data['Drug Type'] != "9:Inorganic"]
data = data.loc[data['Phase'] >= 4]

In [4]:
data = data[['Parent Molecule', 'Synonyms', 'Phase', 'ATC Codes', 'Level 4 ATC Codes', 'Level 3 ATC Codes', 'Level 2 ATC Codes', 'Level 1 ATC Codes',
            'Drug Type', 'Passes Rule of Five', 'Chirality', 'Prodrug', 'Oral', 'Parenteral', 'Topical', 'Black Box', 'Availability Type', 'Withdrawn Year',
            'Withdrawn Reason', 'Withdrawn Country', 'Withdrawn Class', 'Smiles']]

In [5]:
data.rename(columns = {'Parent Molecule': 'chembl_id',
                      'Synonyms': 'synonyms',
                      'Phase': 'max_phase',
                      'ATC Codes': 'atc_codes',
                      'Level 4 ATC Codes': 'level_4_atc',
                      'Level 3 ATC Codes': 'level_3_atc',
                      'Level 2 ATC Codes': 'level_2_atc',
                      'Level 1 ATC Codes': 'levle_1_atc',
                      'Drug Type': 'drug_type',
                      'Passes Rule of Five': 'passes_ro5',
                      'Chirality': 'chirality',
                      'Prodrug': 'prodrug',
                      'Oral': 'oral',
                      'Parenteral': 'parenteral',
                      'Topical': 'topical',
                      'Black Box': 'black_box',
                      'Availability Type': 'availability_type',
                      'Withdrawn Year': 'withdrawn_year',
                      'Withdrawn Reason': 'withdrawn_reason',
                      'Withdrawn Country': 'withdrawn_country',
                      'Withdrawn Class': 'withdrawn_class',
                      'Smiles': 'smiles'}, inplace=True)

In [6]:
list_of_features = ['molecule_type', 'structure_type', 'therapeutic_flag']
list_of_properties = ['alogp', 'aromatic_rings', 'cx_logd', 'cx_logp', 'cx_most_apka', 'cx_most_bpka', 'full_mwt', 'hba', 'hba_lipinski',
                     'hbd', 'hbd_lipinski', 'heavy_atoms', 'molecular_species', 'mw_freebase', 'mw_monoisotopic', 'num_lipinski_ro5_violations',
                     'num_ro5_violations', 'psa', 'qed_weighted', 'ro3_pass', 'rtb']

In [7]:
molecule = new_client.molecule

In [8]:
data['smiles'] = data['smiles'].fillna('missing')

In [9]:
for i in list_of_features:
    data[i] = 'missing'
for i in list_of_properties:
    data[i] = 'missing'
    
data['parent_chembl_id'] = 'missing'
data['parent_smiles'] = 'missing'
data['inchi_key'] = 'missing'
data['parent_inchi_key'] = 'missing'

In [10]:
for mol in tqdm(list(data['chembl_id']), position=0, leave=True):
    try:
        res = molecule.get(mol)

        # add features
        if mol == res['molecule_chembl_id']:
            index = data.loc[data['chembl_id'] == mol].index.values[0]
            for feature in list_of_features:
                if feature == 'atc_classifications' and not res[feature]:
                    data.at[index, feature] = None
                elif feature == 'atc_classifications' and res[feature]:
                    data.at[index, feature] = '+'.join(res[feature])
                else:
                    data.at[index, feature] = res[feature]

            # add properties       
            try:
                for prop in list_of_properties:
                    data.at[index, prop] = res['molecule_properties'][prop]
            except TypeError:
                print('No properties for molecule {}'.format(mol))

            # try to add smiles if exists
            try:
                if data.loc[data['chembl_id'] == mol]['smiles'].values[0] == 'missing':
                    smiles = res['molecule_structures']['canonical_smiles']
                    inchi_key = res['molecule_structures']['standard_inchi_key']
                    data.at[index, 'canonical_smiles'] = smiles
                    data.at[index, 'inchi_key'] = inchi_key
                else:
                    inchi_key = res['molecule_structures']['standard_inchi_key']
                    data.at[index, 'inchi_key'] = inchi_key
            except TypeError:
                print('No smiles for molecule {}'.format(mol))

            #add parent molecule 
            try:
                parent_mol = res['molecule_hierarchy']['parent_chembl_id']
                data.at[index, 'parent_chembl_id'] = parent_mol
            except TypeError:
                continue
            #check if parent has smiles
            try:
                parent_res = molecule.get(parent_mol)
                parent_smiles = parent_res['molecule_structures']['canonical_smiles']
                parent_inchi_key = parent_res['molecule_structures']['standard_inchi_key']
                data.at[index, 'parent_smiles'] = parent_smiles
                data.at[index, 'parent_inchi_key'] = parent_inchi_key
            except:
                print('No parent smiles for molecule {}'.format(mol))
            
    except Exception as e:
            print(e)
            print("Molecule {} doesn't exist".format(mol))

  1%|          | 26/2260 [00:00<00:17, 125.36it/s]

No smiles for molecule CHEMBL1201301
No parent smiles for molecule CHEMBL1201301
No properties for molecule CHEMBL1201466
No smiles for molecule CHEMBL1201466
No parent smiles for molecule CHEMBL1201466
No properties for molecule CHEMBL1201649
No smiles for molecule CHEMBL1201649
No parent smiles for molecule CHEMBL1201649
No smiles for molecule CHEMBL2110554
No smiles for molecule CHEMBL2110563
No parent smiles for molecule CHEMBL2110563
No smiles for molecule CHEMBL2110565
No smiles for molecule CHEMBL2110570
No smiles for molecule CHEMBL2110572
No smiles for molecule CHEMBL1200640
No parent smiles for molecule CHEMBL1200640
No smiles for molecule CHEMBL1366
No parent smiles for molecule CHEMBL1366
No smiles for molecule CHEMBL306043
No parent smiles for molecule CHEMBL306043
No properties for molecule CHEMBL1201441
No smiles for molecule CHEMBL1201441
No parent smiles for molecule CHEMBL1201441
No properties for molecule CHEMBL1201467
No smiles for molecule CHEMBL1201467
No parent s

  2%|▏         | 53/2260 [00:00<00:17, 123.27it/s]

No smiles for molecule CHEMBL3833364
No parent smiles for molecule CHEMBL3833364
No smiles for molecule CHEMBL3833381
No parent smiles for molecule CHEMBL3833381
No properties for molecule CHEMBL2109005
No smiles for molecule CHEMBL2109005
No parent smiles for molecule CHEMBL2109005
No properties for molecule CHEMBL2109152
No smiles for molecule CHEMBL2109152
No parent smiles for molecule CHEMBL2109152


  4%|▎         | 81/2260 [00:00<00:17, 124.28it/s]

...

 45%|████▍     | 1013/2260 [00:10<00:12, 102.86it/s]

No smiles for molecule CHEMBL2110566
No smiles for molecule CHEMBL2110555
No smiles for molecule CHEMBL2110560
No smiles for molecule CHEMBL2110573
No properties for molecule CHEMBL2108758
No smiles for molecule CHEMBL2108758
No parent smiles for molecule CHEMBL2108758
No smiles for molecule CHEMBL4298110


 46%|████▌     | 1040/2260 [00:10<00:10, 115.73it/s]

No parent smiles for molecule CHEMBL4298110
No smiles for molecule CHEMBL4301175


100%|██████████| 2260/2260 [00:22<00:00, 98.92it/s] 


In [11]:
data = data.loc[~((data['smiles'] == 'missing') & (data['parent_smiles'] == 'missing'))]

In [19]:
data.columns

Index(['chembl_id', 'synonyms', 'max_phase', 'atc_codes', 'level_4_atc',
       'level_3_atc', 'level_2_atc', 'levle_1_atc', 'drug_type', 'passes_ro5',
       'chirality', 'prodrug', 'oral', 'parenteral', 'topical', 'black_box',
       'availability_type', 'withdrawn_year', 'withdrawn_reason',
       'withdrawn_country', 'withdrawn_class', 'smiles', 'molecule_type',
       'structure_type', 'therapeutic_flag', 'alogp', 'aromatic_rings',
       'cx_logd', 'cx_logp', 'cx_most_apka', 'cx_most_bpka', 'full_mwt', 'hba',
       'hba_lipinski', 'hbd', 'hbd_lipinski', 'heavy_atoms',
       'molecular_species', 'mw_freebase', 'mw_monoisotopic',
       'num_lipinski_ro5_violations', 'num_ro5_violations', 'psa',
       'qed_weighted', 'ro3_pass', 'rtb', 'parent_chembl_id', 'parent_smiles',
       'inchi_key', 'parent_inchi_key'],
      dtype='object')

In [35]:
toxic_carcinogenic = ['CHEMBL103','CHEMBL1200430','CHEMBL1200686','CHEMBL1200973',
 'CHEMBL1201314', 'CHEMBL1201572', 'CHEMBL1201581', 'CHEMBL1201866',
 'CHEMBL1220', 'CHEMBL135', 'CHEMBL137', 'CHEMBL1393', 'CHEMBL1456',
 'CHEMBL1479', 'CHEMBL1511', 'CHEMBL152', 'CHEMBL1542', 'CHEMBL1554',
 'CHEMBL160', 'CHEMBL1643', 'CHEMBL1651906', 'CHEMBL1742990', 'CHEMBL182',
 'CHEMBL2107841', 'CHEMBL2107857', 'CHEMBL2108027', 'CHEMBL2108078',
 'CHEMBL2108724', 'CHEMBL221959', 'CHEMBL269732', 'CHEMBL3301581',
 'CHEMBL34259', 'CHEMBL413', 'CHEMBL414357', 'CHEMBL416', 'CHEMBL417',
 'CHEMBL467', 'CHEMBL476', 'CHEMBL494753', 'CHEMBL515', 'CHEMBL53463',
 'CHEMBL671', 'CHEMBL717', 'CHEMBL83', 'CHEMBL866']
toxic_cardio = ['CHEMBL118', 'CHEMBL121', 'CHEMBL479', 'CHEMBL1020', 'CHEMBL103', 'CHEMBL1070',
 'CHEMBL1071', 'CHEMBL1098', 'CHEMBL1108', 'CHEMBL1117', 'CHEMBL11359',
 'CHEMBL1164729', 'CHEMBL1171837', 'CHEMBL1200430', 'CHEMBL1200973',
 'CHEMBL1201336', 'CHEMBL1201577', 'CHEMBL1201824', 'CHEMBL1276308',
 'CHEMBL1294', 'CHEMBL1297', 'CHEMBL13', 'CHEMBL134', 'CHEMBL135', 'CHEMBL139',
 'CHEMBL1431', 'CHEMBL1511', 'CHEMBL154', 'CHEMBL154111', 'CHEMBL157101',
 'CHEMBL15770', 'CHEMBL16', 'CHEMBL1655', 'CHEMBL1663', 'CHEMBL1743047',
 'CHEMBL1760', 'CHEMBL178', 'CHEMBL2007641', 'CHEMBL2108676', 'CHEMBL24',
 'CHEMBL24828', 'CHEMBL27', 'CHEMBL405', 'CHEMBL417', 'CHEMBL42', 'CHEMBL469',
 'CHEMBL471', 'CHEMBL473', 'CHEMBL494753', 'CHEMBL499', 'CHEMBL517',
 'CHEMBL521', 'CHEMBL527', 'CHEMBL533', 'CHEMBL53463', 'CHEMBL558', 'CHEMBL563',
 'CHEMBL571', 'CHEMBL58', 'CHEMBL595', 'CHEMBL599', 'CHEMBL6', 'CHEMBL622',
 'CHEMBL633', 'CHEMBL640', 'CHEMBL649', 'CHEMBL651', 'CHEMBL652', 'CHEMBL686',
 'CHEMBL717', 'CHEMBL802', 'CHEMBL898',
]
toxic_dermis = ['CHEMBL108', 'CHEMBL1200431', 'CHEMBL1201589', 'CHEMBL1201824', 'CHEMBL1366',
 'CHEMBL2108676', 'CHEMBL34259', 'CHEMBL428647', 'CHEMBL57', 'CHEMBL741',
 'CHEMBL92',
]
toxic_gastro = ['CHEMBL118', 'CHEMBL1020', 'CHEMBL1070', 'CHEMBL1071', 'CHEMBL109',
 'CHEMBL1110', 'CHEMBL11359', 'CHEMBL1201289', 'CHEMBL1297', 'CHEMBL1351',
 'CHEMBL139', 'CHEMBL1447', 'CHEMBL1460', 'CHEMBL154', 'CHEMBL154111',
 'CHEMBL15770', 'CHEMBL1651906', 'CHEMBL1742982', 'CHEMBL1753',
 'CHEMBL2108676', 'CHEMBL2216870', 'CHEMBL3184512', 'CHEMBL34259', 'CHEMBL469',
 'CHEMBL481', 'CHEMBL521', 'CHEMBL527', 'CHEMBL550348', 'CHEMBL563',
 'CHEMBL571', 'CHEMBL599', 'CHEMBL6', 'CHEMBL622', 'CHEMBL64', 'CHEMBL686',
 'CHEMBL803', 'CHEMBL898', 'CHEMBL92', 
]
toxic_hema = ['CHEMBL1024', 'CHEMBL105', 'CHEMBL108', 'CHEMBL1094', 'CHEMBL1096882',
 'CHEMBL1117', 'CHEMBL11359', 'CHEMBL1161681', 'CHEMBL1201281',
 'CHEMBL1201314', 'CHEMBL1276308', 'CHEMBL129', 'CHEMBL1351', 'CHEMBL1366',
 'CHEMBL152', 'CHEMBL1542', 'CHEMBL1643', 'CHEMBL1651906', 'CHEMBL170',
 'CHEMBL178', 'CHEMBL182', 'CHEMBL34259', 'CHEMBL38', 'CHEMBL417', 'CHEMBL42',
 'CHEMBL428647','CHEMBL44657', 'CHEMBL467', 'CHEMBL476', 'CHEMBL481',
 'CHEMBL514', 'CHEMBL515', 'CHEMBL53463', 'CHEMBL553025', 'CHEMBL58',
 'CHEMBL640', 'CHEMBL671', 'CHEMBL70927','CHEMBL803', 'CHEMBL820', 'CHEMBL833',
 'CHEMBL84', 'CHEMBL852', 'CHEMBL92']
toxic_hepa = ['CHEMBL1077', 'CHEMBL1201506', 'CHEMBL1324', 'CHEMBL109', 'CHEMBL1094',
 'CHEMBL112', 'CHEMBL1131', 'CHEMBL1171837', 'CHEMBL1200436', 'CHEMBL1201187',
 'CHEMBL1201288', 'CHEMBL1237044', 'CHEMBL129', 'CHEMBL1380', 'CHEMBL141',
 'CHEMBL1460', 'CHEMBL1479', 'CHEMBL1518', 'CHEMBL1538', 'CHEMBL157101',
 'CHEMBL1651906', 'CHEMBL2107825', 'CHEMBL2108611', 'CHEMBL2216870',
 'CHEMBL222559', 'CHEMBL34259', 'CHEMBL344159', 'CHEMBL354541', 'CHEMBL461101',
 'CHEMBL471', 'CHEMBL476', 'CHEMBL53463', 'CHEMBL535', 'CHEMBL550348',
 'CHEMBL558', 'CHEMBL57', 'CHEMBL623', 'CHEMBL633', 'CHEMBL64', 'CHEMBL713',
 'CHEMBL803', 'CHEMBL806', 'CHEMBL885', 'CHEMBL92', 'CHEMBL922', 'CHEMBL957',
 'CHEMBL960', 'CHEMBL973', 'CHEMBL991']
toxic_immune = ['CHEMBL11359', 'CHEMBL1201544', 'CHEMBL1201589', 'CHEMBL1201595',
 'CHEMBL1201824', 'CHEMBL1201826', 'CHEMBL1201837', 'CHEMBL1237025',
 'CHEMBL1351', 'CHEMBL1374379', 'CHEMBL1380', 'CHEMBL1456', 'CHEMBL1460',
 'CHEMBL1542', 'CHEMBL1550', 'CHEMBL160', 'CHEMBL1742990', 'CHEMBL2108676',
 'CHEMBL269732', 'CHEMBL3137342', 'CHEMBL3544926', 'CHEMBL413', 'CHEMBL414804',
 'CHEMBL428647', 'CHEMBL469', 'CHEMBL57', 'CHEMBL643', 'CHEMBL852', 'CHEMBL866'
 'CHEMBL92',    
]
infections = ['CHEMBL479', 'CHEMBL1201289', 'CHEMBL1201572', 'CHEMBL1201581',
 'CHEMBL1201607', 'CHEMBL1201828', 'CHEMBL1276308', 'CHEMBL1447', 'CHEMBL1456',
 'CHEMBL160', 'CHEMBL1742990', 'CHEMBL1753', 'CHEMBL2107857', 'CHEMBL2216870',
 'CHEMBL221959', 'CHEMBL269732', 'CHEMBL3184512', 'CHEMBL34259', 'CHEMBL413',
 'CHEMBL553025', 'CHEMBL643', 'CHEMBL866',
]
toxic_metabolism = ['CHEMBL129', 'CHEMBL1380', 'CHEMBL141', 'CHEMBL1429', 'CHEMBL1431',
 'CHEMBL1460', 'CHEMBL1743047', 'CHEMBL1760', 'CHEMBL585', 'CHEMBL713',
 'CHEMBL922', 'CHEMBL945', 'CHEMBL991',   
]
toxic_muscoskelet = ['CHEMBL129', 'CHEMBL32', 'CHEMBL33', 'CHEMBL4', 'CHEMBL640', 'CHEMBL701',
 'CHEMBL717', 'CHEMBL8']

toxic_nephro = ['CHEMBL1024', 'CHEMBL105', 'CHEMBL1072', 'CHEMBL11359', 'CHEMBL1200558',
 'CHEMBL1366', 'CHEMBL152', 'CHEMBL160', 'CHEMBL1651906', 'CHEMBL1747',
 'CHEMBL177', 'CHEMBL2105720', 'CHEMBL3039597', 'CHEMBL34259', 'CHEMBL3989769',
 'CHEMBL507870', 'CHEMBL550348', 'CHEMBL666', 'CHEMBL922']

toxic_neuro = ['CHEMBL118', 'CHEMBL646', 'CHEMBL1020', 'CHEMBL1024', 'CHEMBL103',
 'CHEMBL1070', 'CHEMBL1071', 'CHEMBL1096882', 'CHEMBL1098', 'CHEMBL11359',
 'CHEMBL1171837', 'CHEMBL1200430', 'CHEMBL1200973', 'CHEMBL1201112',
 'CHEMBL1297', 'CHEMBL1342', 'CHEMBL135', 'CHEMBL139', 'CHEMBL1479',
 'CHEMBL1511', 'CHEMBL154', 'CHEMBL154111', 'CHEMBL15770', 'CHEMBL1619',
 'CHEMBL1747', 'CHEMBL177', 'CHEMBL1795072', 'CHEMBL3039597', 'CHEMBL3137342',
 'CHEMBL32', 'CHEMBL33', 'CHEMBL3989738', 'CHEMBL3989769', 'CHEMBL4',
 'CHEMBL407', 'CHEMBL42', 'CHEMBL469', 'CHEMBL494753', 'CHEMBL521', 'CHEMBL527',
 'CHEMBL539697', 'CHEMBL563', 'CHEMBL571', 'CHEMBL599', 'CHEMBL6', 'CHEMBL622',
 'CHEMBL666', 'CHEMBL686', 'CHEMBL701', 'CHEMBL717', 'CHEMBL8', 'CHEMBL81',
 'CHEMBL83', 'CHEMBL86', 'CHEMBL898'
    ]
toxic_psyche = ['CHEMBL646', 'CHEMBL1089', 'CHEMBL11', 'CHEMBL1112', 'CHEMBL1113',
 'CHEMBL1118', 'CHEMBL1175', 'CHEMBL117785', 'CHEMBL1200854', 'CHEMBL1200986',
 'CHEMBL1201', 'CHEMBL1201168', 'CHEMBL1237021', 'CHEMBL14376', 'CHEMBL1508',
 'CHEMBL1621', 'CHEMBL1628227', 'CHEMBL1795072' ,'CHEMBL2104993',
 'CHEMBL2105760', 'CHEMBL21731', 'CHEMBL259209', 'CHEMBL3989843', 'CHEMBL41',
 'CHEMBL415', 'CHEMBL416956', 'CHEMBL439849', 'CHEMBL445', 'CHEMBL490',
 'CHEMBL549', 'CHEMBL567', 'CHEMBL621', 'CHEMBL623', 'CHEMBL629', 'CHEMBL637',
 'CHEMBL644', 'CHEMBL654', 'CHEMBL668', 'CHEMBL701', 'CHEMBL716', 'CHEMBL72',
 'CHEMBL809', 'CHEMBL814', 'CHEMBL894', 'CHEMBL972', 'CHEMBL99946',
]
toxic_respiratory = ['CHEMBL656', 'CHEMBL103', 'CHEMBL11359', 'CHEMBL1200973', 'CHEMBL1201589',
 'CHEMBL1201776', 'CHEMBL1201824', 'CHEMBL1201826', 'CHEMBL1237044',
 'CHEMBL1263', 'CHEMBL1274', 'CHEMBL1342', 'CHEMBL135', 'CHEMBL1374379',
 'CHEMBL1431', 'CHEMBL1511', 'CHEMBL1643', 'CHEMBL177', 'CHEMBL2108676',
 'CHEMBL2216870', 'CHEMBL33986', 'CHEMBL34259', 'CHEMBL398707', 'CHEMBL403664',
 'CHEMBL469', 'CHEMBL494753', 'CHEMBL511142', 'CHEMBL592', 'CHEMBL596',
 'CHEMBL607', 'CHEMBL633', 'CHEMBL643', 'CHEMBL651', 'CHEMBL655', 'CHEMBL70',
 'CHEMBL717', 'CHEMBL780', 'CHEMBL81', 'CHEMBL83', 'CHEMBL895', 'CHEMBL92',
 'CHEMBL963']
toxic_teratogen = ['CHEMBL1014', 'CHEMBL1017', 'CHEMBL1023', 'CHEMBL1069', 'CHEMBL109',
 'CHEMBL1111', 'CHEMBL1131', 'CHEMBL1165', 'CHEMBL1168', 'CHEMBL1200692',
 'CHEMBL1201314', 'CHEMBL1237', 'CHEMBL125', 'CHEMBL1456', 'CHEMBL1479',
 'CHEMBL1513', 'CHEMBL1519', 'CHEMBL152', 'CHEMBL1542' ,'CHEMBL1554',
 'CHEMBL1560', 'CHEMBL1581', 'CHEMBL1592', 'CHEMBL1619', 'CHEMBL1639',
 'CHEMBL1643', 'CHEMBL1747', 'CHEMBL182', 'CHEMBL191', 'CHEMBL2007641',
 'CHEMBL2028661', 'CHEMBL2103873', 'CHEMBL2105737', 'CHEMBL2107834',
 'CHEMBL262777', 'CHEMBL3039597', 'CHEMBL3039598', 'CHEMBL34259', 'CHEMBL38',
 'CHEMBL473417', 'CHEMBL476', 'CHEMBL507870', 'CHEMBL515', 'CHEMBL547',
 'CHEMBL577', 'CHEMBL578', 'CHEMBL606', 'CHEMBL717', 'CHEMBL813', 'CHEMBL838',
 'CHEMBL866', 'CHEMBL957', 'CHEMBL960', 'CHEMBL973',
]
toxic_vascular = ['CHEMBL103', 'CHEMBL11359', 'CHEMBL1161681', 'CHEMBL1171837', 'CHEMBL1200430',
 'CHEMBL1200973', 'CHEMBL1201202', 'CHEMBL1201212', 'CHEMBL1201336',
 'CHEMBL1201438', 'CHEMBL1201476', 'CHEMBL1201589', 'CHEMBL1201651',
 'CHEMBL1201772', 'CHEMBL1201824', 'CHEMBL1201826', 'CHEMBL134', 'CHEMBL135',
 'CHEMBL136478', 'CHEMBL1374379', 'CHEMBL1431', 'CHEMBL1464', 'CHEMBL1479',
 'CHEMBL1511', 'CHEMBL1550', 'CHEMBL16', 'CHEMBL160', 'CHEMBL1742982',
 'CHEMBL1760', 'CHEMBL198362', 'CHEMBL2103827', 'CHEMBL2108676',
 'CHEMBL221959', 'CHEMBL231779', 'CHEMBL34259', 'CHEMBL42', 'CHEMBL428647',
 'CHEMBL469', 'CHEMBL481', 'CHEMBL494753', 'CHEMBL512351', 'CHEMBL53463',
 'CHEMBL539697', 'CHEMBL550348', 'CHEMBL643', 'CHEMBL717', 'CHEMBL81',
 'CHEMBL83', 'CHEMBL92']

In [36]:
toxic_dict = {"toxic_carcinogenic": toxic_carcinogenic, "toxic_cardio": toxic_cardio, "toxic_dermis": toxic_dermis, "toxic_gastro": toxic_gastro,
              "toxic_hema": toxic_hema, "toxic_hepa": toxic_hepa, "toxic_immune": toxic_immune, "infections": infections, 
              "toxic_metabolism": toxic_metabolism, "toxic_muscoskelet": toxic_muscoskelet, "toxic_nephro": toxic_nephro, 
              "toxic_neuro": toxic_neuro, "toxic_psyche": toxic_psyche, "toxic_respiratory": toxic_respiratory, 
              "toxic_teratogen": toxic_teratogen, "toxic_vascular": toxic_vascular}

In [ ]:
data['chembl_tox'] = 'Safe'
for effect in toxic_dict:
    for molecule in toxic_dict[effect]:
        try:
            old_label = data.loc[data['chembl_id'] == molecule]['chembl_tox'].values[0]
        except IndexError:
            print('Mol {} not in the set'.format(molecule))
        if old_label == 'Safe':
            data.loc[data['chembl_id'] == molecule, 'chembl_tox'] = effect
        else:
            data.loc[data['chembl_id'] == molecule, 'chembl_tox'] = '{}+{}'.format(old_label, effect)

In [96]:
data['action_type'] = 'missing'
data['direct_interaction'] = 'missing'
data['disease_efficacy'] = 'missing'
data['molecular_mechanism'] = 'missing'
data['target_chembl_id'] = 'missing'

In [91]:
mechanism_list = ['action_type', 'direct_interaction', 'disease_efficacy', 'molecular_mechanism', 'target_chembl_id']

In [98]:
for molecule in tqdm(list(data['chembl_id'])):
    for mechanism in new_client.mechanism.filter(molecule_chembl_id=molecule):
        if mechanism['max_phase'] == 4:
            for i in mechanism_list:
                if data.loc[data['chembl_id'] == molecule][i].values[0] == 'missing':
                    data.loc[data['chembl_id'] == molecule, i] = mechanism[i]
                else:
                    old_label = data.loc[data['chembl_id'] == molecule][i].values[0]
                    data.loc[data['chembl_id'] == molecule, i] = '{}+{}'.format(old_label, mechanism[i])

100%|██████████| 2164/2164 [20:37<00:00,  1.75it/s]


In [144]:
data['pubchem_cid'] = 'missing'
for i in tqdm(list(data['inchi_key'])):
    try:
        cid = pcp.get_cids(i, "inchikey")[0]
        data.loc[data['inchi_key'] == i, 'pubchem_cid'] = cid
    except:
        print("Molecule not found")

  1%|          | 20/2164 [00:11<20:14,  1.77it/s]

Molecule not found


  1%|          | 21/2164 [00:11<20:20,  1.76it/s]

Molecule not found


  4%|▍         | 90/2164 [00:56<22:29,  1.54it/s]

...

 99%|█████████▉| 2139/2164 [21:42<00:14,  1.67it/s]

Molecule not found


100%|█████████▉| 2162/2164 [22:02<00:01,  1.55it/s]

Molecule not found


100%|██████████| 2164/2164 [22:03<00:00,  1.63it/s]


In [118]:
data['withdrawn'] = 0

In [119]:
data.loc[data['availability_type'] == 'Withdrawn', 'withdrawn'] = 1

In [147]:
data.to_csv('/home/dionizije/Documents/DAO/data/not_from_script/chembl_4_full.csv')

In [291]:
data[['chembl_id', 'pubchem_cid', 'smiles', 'parent_smiles', 'chembl_tox', 'withdrawn']].to_csv('/home/dionizije/Documents/DAO/data/not_from_script/chembl_4_smiles.csv')

# Process DrugBank data

In [272]:
drugbank = pd.read_csv('structure links.csv')

In [273]:
drugbank = drugbank.loc[~drugbank['InChIKey'].isin(data['inchi_key'])]

In [274]:
drugbank = drugbank[['DrugBank ID', 'InChIKey', 'Drug Groups', 'SMILES', 'Name']]

In [275]:
drugbank.rename(columns = {'DrugBank ID': 'drugbank_id',
                          'InChIKey': 'inchi_key',
                          'Drug Groups': 'drug_groups',
                          'SMILES': 'smiles',
                          'Name': 'synonyms'}, inplace=True)

In [276]:
drugbank_list_of_features = ['molecule_type', 'structure_type', 'therapeutic_flag', 'molecule_chembl_id', 'max_phase', 'atc_classifications', 
                             'chirality', 'prodrug', 'oral', 'parenteral', 'topical', 'black_box_warning', 'availability_type', 'withdrawn_year',
                    'withdrawn_reason', 'withdrawn_country', 'withdrawn_class', 'molecule_type', 'structure_type', 'therapeutic_flag']
drugbank_list_of_properties = ['alogp', 'aromatic_rings', 'cx_logd', 'cx_logp', 'cx_most_apka', 'cx_most_bpka', 'full_mwt', 'hba', 'hba_lipinski',
                             'hbd', 'hbd_lipinski', 'heavy_atoms', 'molecular_species', 'mw_freebase', 'mw_monoisotopic', 'num_lipinski_ro5_violations',
                             'num_ro5_violations', 'psa', 'qed_weighted', 'ro3_pass', 'rtb']

In [277]:
for i in drugbank_list_of_features:
    drugbank[i] = 'missing'
for i in list_of_properties:
    drugbank[i] = 'missing'

In [278]:
drugbank['parent_chembl_id'] = 'missing'
drugbank['parent_smiles'] = 'missing'
drugbank['parent_inchi_key'] = 'missing'

In [279]:
for mol in tqdm(list(drugbank['inchi_key']), position=0, leave=True):
    try:
        res = new_client.molecule.get(mol)

        # add features
        if mol == res['molecule_structures']['standard_inchi_key']:
            index = drugbank.loc[drugbank['inchi_key'] == mol].index.values[0]
            for feature in drugbank_list_of_features:
                if feature == 'atc_classifications' and not res[feature]:
                    drugbank.at[index, feature] = None
                elif feature == 'atc_classifications' and res[feature]:
                    drugbank.at[index, feature] = '+'.join(res[feature])
                else:
                    drugbank.at[index, feature] = res[feature]

            # add properties
            try:
                for prop in drugbank_list_of_properties:
                    drugbank.at[index, prop] = res['molecule_properties'][prop]
            except TypeError:
                print('No properties for molecule {}'.format(mol))

            # try to add smiles if exists
            try:
                if drugbank.loc[drugbank['inchi_key'] == mol]['smiles'].values[0] == 'missing':
                    smiles = res['molecule_structures']['canonical_smiles']
                    drugbank.at[index, 'canonical_smiles'] = smiles
            except TypeError:
                print('No smiles for molecule {}'.format(mol))

            # add parent molecule
            try:
                parent_mol = res['molecule_hierarchy']['parent_chembl_id']
                drugbank.at[index, 'parent_chembl_id'] = parent_mol
            except TypeError:
                continue
            # check if parent has smiles
            try:
                parent_res = molecule.get(parent_mol)
                parent_smiles = parent_res['molecule_structures']['canonical_smiles']
                parent_inchi_key = parent_res['molecule_structures']['standard_inchi_key']
                drugbank.at[index, 'parent_smiles'] = parent_smiles
                drugbank.at[index, 'parent_inchi_key'] = parent_inchi_key
            except:
                print('No parent smiles for molecule {}'.format(mol))

    except Exception as e:
        print(e)
        print("Molecule {} doesn't exist".format(mol))

  0%|          | 0/887 [00:00<?, ?it/s]

No parent smiles for molecule OIRCOABEOLEUMC-GEJPAHFPSA-N
No parent smiles for molecule GFIJNRVAKGFPGQ-LIJARHBVSA-N
No parent smiles for molecule BLCLNMBMMGCOAS-URPVMXJPSA-N
No parent smiles for molecule NDAYQJDHGXTBJL-MWWSRJDJSA-N
No parent smiles for molecule NFLWUMRGJYTJIN-PNIOQBSNSA-N
No parent smiles for molecule SBNPWPIBESPSIF-MHWMIDJBSA-N


  1%|          | 7/887 [00:12<26:44,  1.82s/it]

HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/JLTCWSBVQSZVLT-UHFFFAOYSA-N (Caused by ResponseError('too many 404 error responses',))
Molecule JLTCWSBVQSZVLT-UHFFFAOYSA-N doesn't exist


  1%|          | 8/887 [00:25<53:47,  3.67s/it]

...

HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/AWMAOFAHBPCBHJ-YZUKSGEXSA-M (Caused by ResponseError('too many 404 error responses',))
Molecule AWMAOFAHBPCBHJ-YZUKSGEXSA-M doesn't exist


100%|█████████▉| 885/887 [1:31:20<00:14,  7.25s/it]

HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/UXNXMBYCBRBRFD-MUUNZHRXSA-N (Caused by ResponseError('too many 404 error responses',))
Molecule UXNXMBYCBRBRFD-MUUNZHRXSA-N doesn't exist


100%|██████████| 887/887 [1:31:33<00:00,  6.19s/it]

HTTPSConnectionPool(host='www.ebi.ac.uk', port=443): Max retries exceeded with url: /chembl/api/data/molecule/AEBYHKKMCWUMKX-LNTZDJBBSA-K (Caused by ResponseError('too many 404 error responses',))
Molecule AEBYHKKMCWUMKX-LNTZDJBBSA-K doesn't exist
No parent smiles for molecule RCGYDFVCAAKKNG-UHFFFAOYSA-N


In [281]:
drugbank['action_type'] = 'missing'
drugbank['direct_interaction'] = 'missing'
drugbank['disease_efficacy'] = 'missing'
drugbank['molecular_mechanism'] = 'missing'
drugbank['target_chembl_id'] = 'missing'

In [283]:
for molecule in tqdm(list(drugbank['molecule_chembl_id'])):
    for mechanism in new_client.mechanism.filter(molecule_chembl_id=molecule):
        if mechanism['max_phase'] == 4:
            for i in mechanism_list:
                if drugbank.loc[drugbank['molecule_chembl_id'] == molecule][i].values[0] == 'missing':
                    drugbank.loc[drugbank['molecule_chembl_id'] == molecule, i] = mechanism[i]
                else:
                    old_label = drugbank.loc[drugbank['molecule_chembl_id'] == molecule][i].values[0]
                    drugbank.loc[drugbank['molecule_chembl_id'] == molecule, i] = '{}+{}'.format(old_label, mechanism[i])

100%|██████████| 887/887 [05:32<00:00,  2.67it/s]


In [315]:
drugbank = drugbank.loc[(drugbank['max_phase'] == 'missing') | (drugbank['max_phase'] == 4)]

In [295]:
drugbank['chembl_tox'] = 'Safe'
for effect in toxic_dict:
    for molecule in toxic_dict[effect]:
        try:
            old_label = drugbank.loc[drugbank['molecule_chembl_id'] == molecule]['chembl_tox'].values[0]
        except IndexError:
            print('Mol {} not in the set'.format(molecule))
        if old_label == 'Safe':
            drugbank.loc[drugbank['molecule_chembl_id'] == molecule, 'chembl_tox'] = effect
        else:
            drugbank.loc[drugbank['molecule_chembl_id'] == molecule, 'chembl_tox'] = '{}+{}'.format(old_label, effect)

Mol CHEMBL103 not in the set
Mol CHEMBL1200430 not in the set
Mol CHEMBL1200686 not in the set
Mol CHEMBL1200973 not in the set
Mol CHEMBL1201314 not in the set
Mol CHEMBL1201572 not in the set
Mol CHEMBL1201581 not in the set
Mol CHEMBL1201866 not in the set
Mol CHEMBL1220 not in the set
Mol CHEMBL135 not in the set
Mol CHEMBL137 not in the set
Mol CHEMBL1393 not in the set
Mol CHEMBL1456 not in the set
Mol CHEMBL1479 not in the set
Mol CHEMBL1511 not in the set
Mol CHEMBL152 not in the set
Mol CHEMBL1542 not in the set
Mol CHEMBL1643 not in the set
Mol CHEMBL1651906 not in the set
Mol CHEMBL1742990 not in the set
Mol CHEMBL182 not in the set
Mol CHEMBL2107841 not in the set
Mol CHEMBL2107857 not in the set
Mol CHEMBL2108027 not in the set
Mol CHEMBL2108078 not in the set
Mol CHEMBL2108724 not in the set
Mol CHEMBL221959 not in the set
Mol CHEMBL269732 not in the set
Mol CHEMBL3301581 not in the set
Mol CHEMBL34259 not in the set
Mol CHEMBL413 not in the set
Mol CHEMBL414357 not in th

In [298]:
drugbank['withdrawn'] = 0

In [302]:
drugbank.loc[drugbank['drug_groups'].str.contains('withdrawn'), 'withdrawn'] = 1

In [308]:
drugbank = drugbank.loc[~drugbank['smiles'].isna()]

In [316]:
drugbank.to_csv('/home/dionizije/Documents/DAO/data/not_from_script/drugbank_4_full.csv')

In [319]:
drugbank[['molecule_chembl_id', 'smiles', 'drug_groups', 'parent_smiles', 'chembl_tox', 'withdrawn']].to_csv('/home/dionizije/Documents/DAO/data/not_from_script/drugbank_4_smiles.csv')